In [1]:
import sys
import os

root_path = ""
for path in os.getcwd().split("\\")[:-2]:
    root_path += f"{path}/"
sys.path.insert(1, root_path)

In [2]:
# Importing stock libraries
import numpy as np
import pandas as pd
import torch
import yaml
import datasets
from torch.utils.data import DataLoader
from transformers import (
    AutoTokenizer,
)

from src.data.preparing import CustomDataset

c:\Users\Admin\OneDrive - Hanoi University of Science and Technology\DANC\source_code\py_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from tqdm import tqdm

def train_each_epoch(epoch, model, device, loader, optimizer):
    logging = {
        "ppl_loss": [],
        "encoder_z_loss": [],
        "encoder_aux_loss": [],
        "decoder_z_loss": [],
        "decoder_aux_loss": [],
    }
    model.train()
    for _, data in (enumerate(tqdm(loader), 0)):
        # labels = data["target_ids"].to(device, dtype=torch.long)
        # labels = model._shift_right(labels)

        # # We set the pad tokens (0) to -100 to be
        # # ignored by the CrossEntropy loss
        # labels = labels.masked_fill_(labels == 0, -100)
        # ids = data["source_ids"].to(device, dtype=torch.long)
        # mask = data["source_mask"].to(device, dtype=torch.long)

        # outputs = model(
        #     input_ids=ids,
        #     attention_mask=mask,
        #     labels=labels,
        #     output_router_logits=True,
        #     return_dict=True,
        # )

        # loss = outputs[0]

        # # logging loss during training model
        # logging["optimize_loss"] = loss.item()
        # logging["encoder_z_loss"] = outputs.encoder_z_loss.item()
        # logging["encoder_aux_loss"] = outputs.encoder_aux_loss.item()
        # logging["decoder_z_loss"] = outputs.decoder_z_loss.item()
        # logging["decoder_aux_loss"] = outputs.decoder_aux_loss.item()

        if _ % 500 == 0:
            print(f"Epoch: {epoch} | Loop:  {_} | Loss:  {0.01}")

        # optimizer.zero_grad()
        # loss.backward()
        # optimizer.step()
    return model, logging

def train(model, device, training_loader, optimizer, num_epochs):
    # Training loop
    all_logging = dict()
    for epoch in range(num_epochs):
        model, logging = train_each_epoch(
            epoch, model, device, training_loader, optimizer
        )
        all_logging[f"epoch_{epoch}"] = logging
    return model, all_logging


In [14]:
with open(os.path.join(root_path, "config/switch_transfomer/config.yaml"), "r") as f:
    config = yaml.safe_load(f)
f.close()
# Set random seeds and deterministic pytorch for reproducibility
torch.manual_seed(config["SEED"])  # pytorch random seed
np.random.seed(config["SEED"])  # numpy random seed
torch.backends.cudnn.deterministic = True

# Prepare data to training
print("Preparing data ...")
dataset = datasets.load_from_disk(os.path.join(root_path, "data/raw/xsum"))
# Creation of Dataset and Dataloader
train_dataset = dataset["train"]
tokenizer = AutoTokenizer.from_pretrained("google/switch-base-8")
training_set = CustomDataset(
    train_dataset, tokenizer, config["MAX_LEN"], config["SUMMARY_LEN"]
)
# # Defining the parameters for creation of dataloaders
# train_params = {
#     "batch_size": config["TRAIN_BATCH_SIZE"],
#     "shuffle": True,
#     "num_workers": 0,
# }
# training_loader = DataLoader(training_set, **train_params)

# # Initial device
# device = "cuda" if cuda.is_available() else "cpu"
# print(f"Train model by {device}")
# # Initial model
# model = SwitchTransformersForConditionalGeneration.from_pretrained(
#     "google/switch-base-8", torch_dtype=torch.bfloat16
# )
# model = model.to(device)
# # Defining the optimizer that will be used to tune the weights of the network in the training session.
# optimizer = torch.optim.Adam(
#     params=model.parameters(), lr=float(config["LEARNING_RATE"])
# )

#  # Start training model
# print("Training model ...")
# model, logging = train(
#     model=model,
#     device=device,
#     training_loader=training_loader,
#     optimizer=optimizer,
#     num_epochs=config["TRAIN_EPOCHS"],
# )
# model.forward()

Preparing data ...


In [8]:
training_set[0]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
c:\Users\Admin\OneDrive - Hanoi University of Science and Technology\DANC\source_code\py_env\Lib\site-packages\transformers\tokenization_utils_base.py:2681: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


{'source_ids': tensor([21603,    10,    37,   423,   583,    13,  1783,    16, 20126, 16496,
             6,    80,    13,     8,   844,  6025,  4161,     6,    19,   341,
           271, 14841,     5,  7057,   161,    19,  4912,    16,  1626,  5981,
            11,   186,  7540,    16,  1276,    15,  2296,     7,  5718,  2367,
         14621,  4161,    57,  4125,   387,     5, 15059,     7,    30,     8,
          4653,  4939,   711,   747,   522, 17879,   788,    12,  1783,    44,
             8, 15763,  6029,  1813,     9,  7472,     5,  1404,  1623,    11,
          5699,   277,   130,  4161,    57, 18368,    16, 20126, 16496,   227,
             8,  2473,  5895,    15,   147,    89, 22411,   139,     8,  1511,
             5,  1485,  3271,     3, 21926,     9,   472, 19623,  5251,     8,
           616,    12, 15614,     8,  1783,     5,    37, 13818, 10564,    15,
            26,     3,     9,     3, 19513,  1481,     6, 18368,   186,  1328,
          2605,    30,  7488,  1887,  

In [20]:
train_dataset[0]

{'document': 'summarize: The full cost of damage in Newton Stewart, one of the areas worst affected, is still being assessed.\nRepair work is ongoing in Hawick and many roads in Peeblesshire remain badly affected by standing water.\nTrains on the west coast mainline face disruption due to damage at the Lamington Viaduct.\nMany businesses and householders were affected by flooding in Newton Stewart after the River Cree overflowed into the town.\nFirst Minister Nicola Sturgeon visited the area to inspect the damage.\nThe waters breached a retaining wall, flooding many commercial properties on Victoria Street - the main shopping thoroughfare.\nJeanette Tate, who owns the Cinnamon Cafe which was badly affected, said she could not fault the multi-agency response once the flood hit.\nHowever, she said more preventative work could have been carried out to ensure the retaining wall did not fail.\n"It is difficult but I do think there is so much publicity for Dumfries and the Nith - and I total

In [25]:
len(training_set[0]["source_ids"])

c:\Users\Admin\OneDrive - Hanoi University of Science and Technology\DANC\source_code\py_env\Lib\site-packages\transformers\tokenization_utils_base.py:2681: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


256